## Инициализация и нормализация

В этом задании вам предстоит реализовать два вида нормализации: по батчам (BatchNorm1d) и по признакам (LayerNorm1d).

In [1]:
from typing import Callable, NamedTuple

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import Tensor

from IPython.display import display

### 1. Реализация BatchNorm1d и LayerNorm1d.

#### 1.1. (2 балла) Реализуйте BatchNorm1d

Подсказка: чтобы хранить текущие значения среднего и дисперсии, вам потребуется метод `torch.nn.Module.register_buffer`, ознакомьтесь с документацией к нему. Подумайте, какие проблемы возникнут, если вы будете просто сохранять ваши значения в тензор.


Важно помнить:
- Понятно, что нормализацию мы добавляем после очередного слоя с параметрами, но до применения функции активации или после? Подумайте, есть ли у одного из этих способов преимущества над другим.
- Модуль нормализации по батчам работает по-разному при обучении и при валидации, и ему нужно понимать, в каком он состоянии. Эта информация доступна в атрибуте модуля `self.training: bool`, его значение определит ветвление логики в вашей реализации метода `forward`.
- Переключение модулей между режимами осуществляется вызовами у объекта модели методов `.train()` (переключение в режим обучения) и `.eval()` (переключение в режим валидации) Почитайте документацию к этим методам. Ваши функции `train_epoch` и `test_epoch` теперь должны переводить модель в нужный режим перед началом обработки данных.

In [2]:
class BatchNorm1d(nn.Module):
    def __init__(
        self, num_features: int, momentum: float = 0.9, eps: float = 1e-5
    ) -> None:
        super().__init__()
        self.scale = nn.Parameter(torch.ones(num_features))
        self.shift = nn.Parameter(torch.zeros(num_features))
        self.register_buffer("running_mean", torch.zeros(num_features))
        self.register_buffer("running_var", torch.ones(num_features))
        self.momentum = momentum
        self.eps = eps

    def forward(self, x: Tensor) -> Tensor:
        print(self.training)
        if self.training:
            mean = torch.mean(x, dim=0)
            var = torch.var(x, dim=0, unbiased=False)
            self.running_mean = (1 - self.momentum) * self.running_mean + self.momentum * mean
            self.running_var = (1 - self.momentum) * self.running_var + self.momentum * var
        else:
            mean = self.running_mean
            var = self.running_var
        x_normalized = self.scale * ((x - mean.view(1, -1)) / torch.sqrt(var.view(1, -1) + self.eps)) + self.shift
        return x_normalized

#### 1.2. (1 балл) Реализуйте LayerNorm1d

Отличия LayerNorm от BatchNorm - в том, что расчёт средних и дисперсий в BatchNorm происходит вдоль размерности батча (см. рисунок слева), а в LayerNorm - вдоль размерности признаков (см. рисунок справа).

<img src="../attachments/norm.png" width="800">

In [3]:
class LayerNorm1d(nn.Module):
    def __init__(self, num_features: int, eps: float = 1e-5) -> None:
        super(LayerNorm1d, self).__init__()
        self.scale = nn.Parameter(torch.ones(num_features))
        self.shift = nn.Parameter(torch.zeros(num_features))
        self.eps = eps

    def forward(self, x: Tensor) -> Tensor:
        mean = torch.mean(x, dim=1, keepdim=True)
        var = torch.var(x, dim=1, keepdim=True, unbiased=False)
        x_normalized = self.scale * ((x - mean) / torch.sqrt(var + self.eps)) + self.shift
        return x_normalized

Замечание: явных тестов на корректность в этом задании нет, так как конкретные реализации могут давать немного разные результаты. Но вы можете проверить корректность своей реализации в эксперименте сами, нормализация должна немного исправлять проблемы неудачной инициализации. Ну и как минимум вы можете вручную проверить, что ваши активации действительно нормализуются в результате применения вашего модуля.

### 2. Эксперименты

В этом задании ваша задача - проверить, какие из приёмов хорошо справляются с нездоровыми активациями в промежуточных слоях. Вам будет дана базовая модель, у которой есть проблемы с инициализацией параметров, попробуйте несколько приёмов для устранения проблем обучения:
1. Хорошая инициализация параметров
2. Ненасыщаемая функция активации (например, `F.leaky_relu`)
3. Нормализация по батчам или по признакам (можно использовать встроенные `nn.BatchNorm1d` и `nn.LayerNorm`)
4. Более продвинутый оптимизатор (`torch.optim.RMSprop`)

#### 2.0. Подготовка: датасет, функции для обучения

Проверять наши гипотезы будем на датасете MNIST, для отладки добавим в функции для обучения возможность использовать только несколько батчей данных

In [4]:
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

train_dataset = datasets.MNIST(
    "data",
    train=True,
    download=True,
    transform=transforms.ToTensor(),
)
test_dataset = datasets.MNIST(
    "data",
    train=False,
    download=True,
    transform=transforms.ToTensor(),
)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [5]:
def training_step(
    batch: tuple[torch.Tensor, torch.Tensor],
    model: nn.Module,
    optimizer: torch.optim.Optimizer,
) -> torch.Tensor:
    # прогоняем батч через модель
    x, y = batch
    logits = model(x)
    # оцениваем значение ошибки
    loss = F.cross_entropy(logits, y)
    # обновляем параметры
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    # возвращаем значение функции ошибки для логирования
    return loss


def train_epoch(
    dataloader: DataLoader,
    model: nn.Module,
    optimizer: torch.optim.Optimizer,
    max_batches: int = 100,
) -> Tensor:
    model.train()
    loss_values: list[float] = []
    for i, batch in enumerate(dataloader):
        loss = training_step(batch, model, optimizer)
        loss_values.append(loss.item())
        if i == max_batches:
            break
    return torch.tensor(loss_values).mean()


@torch.no_grad()
def test_epoch(
    dataloader: DataLoader, model: nn.Module, max_batches: int = 100
) -> Tensor:
    model.eval()
    loss_values: list[float] = []
    for i, batch in enumerate(dataloader):
        x, y = batch
        logits = model(x)
        # оцениваем значение ошибки
        loss = F.cross_entropy(logits, y)
        loss_values.append(loss.item())
        if i == max_batches:
            break
    return torch.tensor(loss_values).mean()

#### 2.1. Определение класса модели (2 балла)

Для удобства проведения экспериментов мы немного усложним создание модели, чтобы можно было задать разные способы инициализации параметров и нормализации промежуточных активаций, не меняя определение класса.

Добавьте в метод `__init__`:
- аргумент, который позволит использовать разные функции активации для промежуточных слоёв
- аргумент, который позволит задавать разные способы нормализации: `None` (без нормализации), `nn.BatchNorm` и `nn.LayerNorm`

In [6]:
def init_std_normal(model: nn.Module) -> None:
    """Функция для инициализации параметров модели стандартным нормальным распределением."""
    for param in model.parameters():
        torch.nn.init.normal_(param.data, mean=0, std=1)

def init_kaiming_normal(model: nn.Module) -> None:
    nn.init.kaiming_normal_(model.fc1.weight)
    nn.init.kaiming_normal_(model.fc2.weight)

from typing import Type


class MLP(nn.Module):
    """Базовая модель для экспериментов

    Args:
        input_dim (int): размерность входных признаков
        hidden_dim (int): размерност скрытого слоя
        output_dim (int): кол-во классов
        act_fn (Callable[[Tensor], Tensor], optional): Функция активации. Defaults to F.tanh.
        init_fn (Callable[[nn.Module], None], optional): Функция для инициализации. Defaults to init_std_normal.
        norm (Type[nn.BatchNorm1d  |  nn.LayerNorm] | None, optional): Способ нормализации промежуточных активаций.
            Defaults to None.
    """
    def __init__(
        self,
        input_dim: int,
        hidden_dim: int,
        output_dim: int,
        act_fn: Callable[[Tensor], Tensor] = F.tanh,
        init_fn: Callable[[nn.Module], None] = init_std_normal,
        norm: Type[nn.BatchNorm1d | nn.LayerNorm] | None = None,
    ) -> None:
        super().__init__()
        # теперь линейные слои будем задавать
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        self.act_fn = act_fn
        self.norm = norm(hidden_dim) if norm else None

        # reinitialize parameters
        init_fn(self)

    def forward(self, x: Tensor) -> Tensor:
        h = self.fc1.forward(x.flatten(1))
        # here you can do normalization
        if self.norm:
            h = self.norm(h)
        return self.fc2.forward(self.act_fn(h))

#### 2.2. Эксперименты (7 баллов)

Проведите по 3 эксперимента с каждой из модификаций с разными значениями `seed`, соберите статистику значений тестовой ошибки после 10 эпох обучения, сделайте выводы о том, что работает лучше

Проверяем:
1. Метод инициализации весов модели: $\mathcal{N}(0, 1)$ / Kaiming normal
2. Функция активации: tanh /  (или любая другая без насыщения)
3. Слой нормализации: None / BatchNorm / LayerNorm
4. Выбранный оптимизатор: SGD / RMSprop / Adam

Итого у нас 7 экспериментов:
- исходный (1)
- смена инициализации (1)
- смена нелинейности (1)
- смена нормализации (2)
- смена оптимизатора (2)

Каждый эксперимент нужно повторить 3 раза с разными значениями random seed, посчитать среднее и вывести результаты в pandas.DataFrame.
Можно дополнительно потестировать разные сочетания опций, например инициализация + нормализация


Чтобы автоматизировать проведение экспериментов, можно использовать функцию, которая будет принимать все необходимые настройки эксперимента, запускать его и сохранять нужные метрики:

In [7]:
def run_experiment(
    model_gen: Callable[[], nn.Module],
    optim_gen: Callable[[nn.Module], torch.optim.Optimizer],
    seed: int,
    n_epochs: int = 10,
    max_batches: int | None = None,
    verbose: bool = False,
) -> float:
    """Функция для запуска экспериментов.

    Args:
        model_gen (Callable[[], nn.Module]): Функция для создания модели
        optim_gen (Callable[[nn.Module], torch.optim.Optimizer]): Функция для создания оптимизатора для модели
        seed (int): random seed
        n_epochs (int, optional): Число эпох обучения. Defaults to 10.
        max_batches (int | None, optional): Если указано, только `max_batches` минибатчей
            будет использоваться при обучении и тестировании. Defaults to None.
        verbose (bool, optional): Выводить ли информацию для отладки. Defaults to False.

    Returns:
        float: Значение ошибки на тестовой выборке в конце обучения
    """
    torch.manual_seed(seed)
    # создадим модель и выведем значение ошибки после инициализации
    model = model_gen()
    optim = optim_gen(model)
    epoch_losses: list[float] = []
    for i in range(n_epochs):
        train_loss = train_epoch(train_loader, model, optim, max_batches=max_batches)
        test_loss = test_epoch(test_loader, model, max_batches=max_batches)
        if verbose:
            print(f"Epoch {i} train loss = {train_loss:.4f}")
            print(f"Epoch {i} test loss = {test_loss:.4f}")

        epoch_losses.append(test_loss.item())

    last_epoch_loss = epoch_losses[-1]
    return last_epoch_loss

Пример использования:

In [8]:
losses = run_experiment(
    model_gen=lambda: MLP(784, 128, 10, init_fn=init_std_normal, norm=None),
    optim_gen=lambda x: torch.optim.SGD(x.parameters(), lr=0.01),
    seed=42,
    n_epochs=10,
    max_batches=100,
    verbose=True,
)

Epoch 0 train loss = 12.6168
Epoch 0 test loss = 9.9327
Epoch 1 train loss = 9.0954
Epoch 1 test loss = 7.5498
Epoch 2 train loss = 6.9607
Epoch 2 test loss = 6.2342
Epoch 3 train loss = 5.8992
Epoch 3 test loss = 5.3655
Epoch 4 train loss = 4.9951
Epoch 4 test loss = 4.7433
Epoch 5 train loss = 4.4778
Epoch 5 test loss = 4.3001
Epoch 6 train loss = 3.9693
Epoch 6 test loss = 3.9605
Epoch 7 train loss = 3.7261
Epoch 7 test loss = 3.6844
Epoch 8 train loss = 3.4223
Epoch 8 test loss = 3.4538
Epoch 9 train loss = 2.9975
Epoch 9 test loss = 3.2638


Для удобства задания настроек эксперимента можно определять их с помощью класса `Experiment`, в котором можно также реализовать логику для строкового представления:

In [9]:
input_dim = 784
hidden_dim = 128
output_dim = len(train_dataset.classes)


class Experiment(NamedTuple):
    init_fn: Callable[[nn.Module], None]
    act_fn: Callable[[Tensor], Tensor]
    norm: Type[nn.BatchNorm1d | nn.LayerNorm] | None
    optim_cls: Type[torch.optim.Optimizer]

    @property
    def model_gen(self) -> Callable[[], nn.Module]:
        return lambda: MLP(
            input_dim, hidden_dim, output_dim, init_fn=self.init_fn, norm=self.norm, act_fn=self.act_fn
        )

    @property
    def optim_gen(self) -> Callable[[nn.Module], torch.optim.Optimizer]:
        return lambda x: self.optim_cls(x.parameters(), lr=0.01)

    def __repr__(self) -> str:
        # TODO: попробуйте сделать представление эксперимента более читаемым
        # representation = ("Метод инициализации весов модели:" + str(self.init_fn.__name__)
        #                     + "\n" + "Функция активации:" + str(self.act_fn.__name__)
        #                     + "\n" + "Слой нормализации:" + str(self.norm.__name__ if self.norm is not None else None)
        #                     + "\n" + "Оптимизатор:" + str(self.optim_cls.__name__) + '\n')
        representation = ('Метод инициализации весов модели: ' + str(self.init_fn.__name__) + ', ' + 'Функция активации: '
                          + str(self.act_fn.__name__) + ', ' + 'Слой нормализации: ' + str(self.norm.__name__ if self.norm is not None else None)
                          + ', ' + 'Оптимизатор: ' + str(self.optim_cls.__name__))
        return representation


Описываем все эксперименты:

In [10]:
options = [
    Experiment(
        init_fn=init_std_normal,
        act_fn=F.tanh,
        norm=None,
        optim_cls=torch.optim.SGD,
    ),
    Experiment(
        init_fn=init_kaiming_normal,
        act_fn=F.tanh,
        norm=None,
        optim_cls=torch.optim.SGD,
    ),
    Experiment(
        init_fn=init_std_normal,
        act_fn=F.silu,
        norm=None,
        optim_cls=torch.optim.SGD,
    ),
    Experiment(
        init_fn=init_std_normal,
        act_fn=F.tanh,
        norm=nn.LayerNorm,
        optim_cls=torch.optim.SGD,
    ),
    Experiment(
        init_fn=init_std_normal,
        act_fn=F.tanh,
        norm=nn.BatchNorm1d,
        optim_cls=torch.optim.SGD,
    ),
    Experiment(
        init_fn=init_std_normal,
        act_fn=F.tanh,
        norm=None,
        optim_cls=torch.optim.RMSprop,
    ),
    Experiment(
        init_fn=init_std_normal,
        act_fn=F.tanh,
        norm=None,
        optim_cls=torch.optim.Adam,
    ),
]

options

[Метод инициализации весов модели: init_std_normal, Функция активации: tanh, Слой нормализации: None, Оптимизатор: SGD,
 Метод инициализации весов модели: init_kaiming_normal, Функция активации: tanh, Слой нормализации: None, Оптимизатор: SGD,
 Метод инициализации весов модели: init_std_normal, Функция активации: silu, Слой нормализации: None, Оптимизатор: SGD,
 Метод инициализации весов модели: init_std_normal, Функция активации: tanh, Слой нормализации: LayerNorm, Оптимизатор: SGD,
 Метод инициализации весов модели: init_std_normal, Функция активации: tanh, Слой нормализации: BatchNorm1d, Оптимизатор: SGD,
 Метод инициализации весов модели: init_std_normal, Функция активации: tanh, Слой нормализации: None, Оптимизатор: RMSprop,
 Метод инициализации весов модели: init_std_normal, Функция активации: tanh, Слой нормализации: None, Оптимизатор: Adam]

Запускаем расчёты:

In [13]:
seeds = [6, 12, 48]  # здесь вам нужно 3 разных значения
results = []

for option in options:
    for seed in seeds:
        loss = run_experiment(
            model_gen=lambda: MLP(input_dim, hidden_dim, output_dim, init_fn=option.init_fn, act_fn=option.act_fn, norm=option.norm),
            optim_gen=lambda x: option.optim_cls(x.parameters()),
            seed=seed,
            n_epochs=10,
            max_batches=None,
            verbose=True,
        )
        results.append([repr(option), seed, loss])

Epoch 0 train loss = 11.7281
Epoch 0 test loss = 8.6669
Epoch 1 train loss = 7.3093
Epoch 1 test loss = 6.1118
Epoch 2 train loss = 5.4957
Epoch 2 test loss = 4.8238
Epoch 3 train loss = 4.4830
Epoch 3 test loss = 4.0405
Epoch 4 train loss = 3.8325
Epoch 4 test loss = 3.5157
Epoch 5 train loss = 3.3787
Epoch 5 test loss = 3.1404
Epoch 6 train loss = 3.0433
Epoch 6 test loss = 2.8587
Epoch 7 train loss = 2.7844
Epoch 7 test loss = 2.6378
Epoch 8 train loss = 2.5767
Epoch 8 test loss = 2.4578
Epoch 9 train loss = 2.4053
Epoch 9 test loss = 2.3110
Epoch 0 train loss = 11.4157
Epoch 0 test loss = 8.5777
Epoch 1 train loss = 7.1368
Epoch 1 test loss = 5.9516
Epoch 2 train loss = 5.2834
Epoch 2 test loss = 4.6240
Epoch 3 train loss = 4.2749
Epoch 3 test loss = 3.8606
Epoch 4 train loss = 3.6567
Epoch 4 test loss = 3.3595
Epoch 5 train loss = 3.2366
Epoch 5 test loss = 3.0058
Epoch 6 train loss = 2.9283
Epoch 6 test loss = 2.7412
Epoch 7 train loss = 2.6911
Epoch 7 test loss = 2.5331
Epoch 8 

Выводим результаты:

In [14]:
import pandas as pd

pd.set_option('display.max_colwidth', None)
df = pd.DataFrame(results)
df = df.set_axis(['options', 'seed', 'loss'], axis=1)
display(df)
print()
df1 = df.groupby('options', as_index=False)['loss'].mean()
df1 = df1.rename(columns = {'loss': 'mean_loss'})
display(df1)

,options,seed,loss
0,"Метод инициализации весов модели: init_std_normal, Функция активации: tanh, Слой нормализации: None, Оптимизатор: SGD",6,2.311021
1,"Метод инициализации весов модели: init_std_normal, Функция активации: tanh, Слой нормализации: None, Оптимизатор: SGD",12,2.225786
2,"Метод инициализации весов модели: init_std_normal, Функция активации: tanh, Слой нормализации: None, Оптимизатор: SGD",48,2.141401
3,"Метод инициализации весов модели: init_kaiming_normal, Функция активации: tanh, Слой нормализации: None, Оптимизатор: SGD",6,0.371709
4,"Метод инициализации весов модели: init_kaiming_normal, Функция активации: tanh, Слой нормализации: None, Оптимизатор: SGD",12,0.364070
5,"Метод инициализации весов модели: init_kaiming_normal, Функция активации: tanh, Слой нормализации: None, Оптимизатор: SGD",48,0.377117
6,"Метод инициализации весов модели: init_std_normal, Функция активации: silu, Слой нормализации: None, Оптимизатор: SGD",6,3.267605
7,"Метод инициализации весов модели: init_std_normal, Функция активации: silu, Слой нормализации: None, Оптимизатор: SGD",12,3.633480
8,"Метод инициализации весов модели: init_std_normal, Функция активации: silu, Слой нормализации: None, Оптимизатор: SGD",48,3.399774
9,"Метод инициализации весов модели: init_std_normal, Функция активации: tanh, Слой нормализации: LayerNorm, Оптимизатор: SGD",6,1.149888


,options,mean_loss
0,"Метод инициализации весов модели: init_kaiming_normal, Функция активации: tanh, Слой нормализации: None, Оптимизатор: SGD",0.370965
1,"Метод инициализации весов модели: init_std_normal, Функция активации: silu, Слой нормализации: None, Оптимизатор: SGD",3.433620
2,"Метод инициализации весов модели: init_std_normal, Функция активации: tanh, Слой нормализации: BatchNorm1d, Оптимизатор: SGD",1.191583
3,"Метод инициализации весов модели: init_std_normal, Функция активации: tanh, Слой нормализации: LayerNorm, Оптимизатор: SGD",1.147906
4,"Метод инициализации весов модели: init_std_normal, Функция активации: tanh, Слой нормализации: None, Оптимизатор: Adam",0.304073
5,"Метод инициализации весов модели: init_std_normal, Функция активации: tanh, Слой нормализации: None, Оптимизатор: RMSprop",0.186117
6,"Метод инициализации весов модели: init_std_normal, Функция активации: tanh, Слой нормализации: None, Оптимизатор: SGD",2.226069


ВЫВОДЫ:

Меньше всего тестовая ошибка при стандартной нормальной инициализации весов, использовании гиперболического тангенса
в качестве функции активации, без использования нормализации данных, с оптимизатором RMSprop. Также можно видеть, 
что с таким методом инициализации весов ошибка не падала резко, что дает надежду на то, что инициализация оптимальна.